**Eksik Veri Doldurma (Data Imputation)**<br/>
*   Eksik öznitelik değerleri olan veri satırlarını (örnekleri) kullanma, onları sil (Oldukça kolay bir çözüm).
*   Eksik öznitelik değerlerini elle doldur (Veri büyüdükçe ve eksik olan verinin önemine göre zaman alıcı ve etkin olmayan bir yönteme dönüşebilir).
*   Eksik öznitelik değerleri için global bir değişken kullan (Null, bilinmiyor).
*   Eksik öznitelik değerlerini ortalama değer ile doldur.
*   Aynı sınıfa ait kayıtların öznitelik değerlerinin ortalaması ile doldur.
*  Olasılığı en fazla olan öznitelik değeriyle doldur.
*   Regresyon yöntemi ile sayısal eksik değerleri tahmin et ve doldur.

In [ ]:
import pandas as pd
import numpy as np

sozluk = {'İsim':pd.Series(['Ada','Cem','Sibel','Ahmet','Mehmet','Ali','Veli',
          'Ayşe','Hüseyin','Necmi','Nalan','Namık']),
          'Meslek':pd.Series(['işçi','işçi','memur','serbest','serbest',None,None,
          'sigortacı','işsiz',None,None,'memur']),
          'Tarih':pd.Series(['11.11.2010','11.11.2010','11.11.2010','18.11.2011','18.11.2011',None,None,
          None,'11.11.2010',None,'18.11.2011','18.11.2011']),          
          'Yaş':pd.Series([21, 24, 25, 44, 31, 27, 35, 33, 42, 29, 41, 43]),
          'ÇocukSayısı':pd.Series([None, None, None, None, None, 1, 2, 0, None, None, None, None]),
          'Puan':pd.Series([89, 87, 77, 55, 70, 79, 73, 79, 54, 92, 61, 69])}
df = pd.DataFrame(sozluk)
print(df)

       İsim     Meslek       Tarih  Yaş  ÇocukSayısı  Puan
0       Ada       işçi  11.11.2010   21          NaN    89
1       Cem       işçi  11.11.2010   24          NaN    87
2     Sibel      memur  11.11.2010   25          NaN    77
3     Ahmet    serbest  18.11.2011   44          NaN    55
4    Mehmet    serbest  18.11.2011   31          NaN    70
5       Ali       None        None   27          1.0    79
6      Veli       None        None   35          2.0    73
7      Ayşe  sigortacı        None   33          0.0    79
8   Hüseyin      işsiz  11.11.2010   42          NaN    54
9     Necmi       None        None   29          NaN    92
10    Nalan       None  18.11.2011   41          NaN    61
11    Namık      memur  18.11.2011   43          NaN    69


**Toplam kaç hücrede eksik değer (NaN ya da None) var?**
Bu sorunun cevabını aşağıdaki kod parçasını çalıştırarak bulabiliriz.

In [ ]:
df.isnull().sum().sum()

17

**Özniteliklerin değer almadığı kaç satır var?**
Aşağıdaki kod satırını çalıştıralım ve görelim.

In [ ]:
df.isnull().sum()

İsim           0
Meslek         4
Tarih          4
Yaş            0
ÇocukSayısı    9
Puan           0
dtype: int64

Kod satırını çalıştırdıktan sonra elde ettiğimiz çıktı yukarıdaki gibidir. Buna göre Meslek özniteliğine sahip olmayan 4 satır, Tarih özniteliğine sahip olmaya 4 satır ve ÇocukSayısı özniteliğine sahip olmayan 9 satır bulunmaktadır.

Eksik değerleri sayısal olarak görmek basit olsa da, eksik değerlerin satır bazında yüzdesini görmek, “bundan sonraki adıma karar vermek adına” daha sağlıklıdır. Bunun için aşağıdaki gibi “eksik_deger_tablosu” isimli ve parametre olarak “dataframe” alan bir fonksiyon yazalım.

In [ ]:
def eksik_deger_tablosu(df): 
    eksik_deger = df.isnull().sum()
    eksik_deger_yuzde = 100 * df.isnull().sum()/len(df)
    eksik_deger_tablo = pd.concat([eksik_deger, eksik_deger_yuzde], axis=1)
    eksik_deger_tablo_son = eksik_deger_tablo.rename(
    columns = {0 : 'Eksik Değerler', 1 : '% Değeri'})
    return eksik_deger_tablo_son
eksik_deger_tablosu(df)

,Eksik Değerler,% Değeri
İsim,0,0.000000
Meslek,4,33.333333
Tarih,4,33.333333
Yaş,0,0.000000
ÇocukSayısı,9,75.000000
Puan,0,0.000000


Burada özellikle %75 eksik değer oranıyla (12 satırın 9'unda bu değer yok) “ÇocukSayısı” özniteliği göze çarpmaktadır.Bu kadar eksik değerin olduğu bir öznitelik büyük ihtimalle işe yaramayacaktır. Bu özniteliği veri kümesinden kaldırmak mantıklı olabilir. Aslında şöyle bir strateji de izleyebiliriz: Belirli bir eşik değerin üzerinde, örneğin %70, eksik değer olan öznitelikleri veri kümesinden çıkar.

In [ ]:
tr = len(df) * .3
df.dropna(thresh = tr, axis = 1, inplace = True)

df

,İsim,Meslek,Tarih,Yaş,Puan
0,Ada,işçi,11.11.2010,21,89
1,Cem,işçi,11.11.2010,24,87
2,Sibel,memur,11.11.2010,25,77
3,Ahmet,serbest,18.11.2011,44,55
4,Mehmet,serbest,18.11.2011,31,70
5,Ali,None,None,27,79
6,Veli,None,None,35,73
7,Ayşe,sigortacı,None,33,79
8,Hüseyin,işsiz,11.11.2010,42,54
9,Necmi,None,None,29,92


Meslek ve Tarih öznitelikleri için farklı eksik değer doldurma stratejileri izlenebilir. Örneğin, Meslek özniteliği olmayan kayıtlara “Diğer” değeri atanabilir. Tarih özniteliği eksik olan kayıtlar, Tarih kolonundaki benzersiz değerlerden ilki ile doldurulabilir 

In [ ]:
#Meslek özniteliğindeki Null değerleri 'Diğer' değeri ile doldur
df['Meslek'] = df['Meslek'].fillna('Diğer')

#Tarih özniteliğindeki Null değerleri Tarih benzersiz değerlerden ilki ile doldur
print(df['Tarih'].unique()[0])
df['Tarih'] = df['Tarih'].fillna(df['Tarih'].unique()[0])

df

11.11.2010


,İsim,Meslek,Tarih,Yaş,Puan
0,Ada,işçi,11.11.2010,21,89
1,Cem,işçi,11.11.2010,24,87
2,Sibel,memur,11.11.2010,25,77
3,Ahmet,serbest,18.11.2011,44,55
4,Mehmet,serbest,18.11.2011,31,70
5,Ali,Diğer,11.11.2010,27,79
6,Veli,Diğer,11.11.2010,35,73
7,Ayşe,sigortacı,11.11.2010,33,79
8,Hüseyin,işsiz,11.11.2010,42,54
9,Necmi,Diğer,11.11.2010,29,92


**Yeni Öznitelikler Oluşturma (Feature Extraction)**<br>
Bazı durumlarda mevcut öznitelikleri kullanarak, “daha fazla işe yarayacağı düşünülen” yeni öznitelikler oluşturulabilir. (1) Örneğin, aşağıdaki kod satırları çalıştırılarak, Geçti isimli yeni bir öznitelik oluşturulmuş ve sınavdan 70 (ve) üzerinde not alan kayıtların bu öznitelik değerleri “True” olarak ayarlanmıştır.

In [ ]:
def basari_durumu(puan):
    return (puan >= 70)

df['Geçti'] = df['Puan'].apply(basari_durumu)
df

,İsim,Meslek,Tarih,Yaş,Puan,Geçti
0,Ada,işçi,11.11.2010,21,89,True
1,Cem,işçi,11.11.2010,24,87,True
2,Sibel,memur,11.11.2010,25,77,True
3,Ahmet,serbest,18.11.2011,44,55,False
4,Mehmet,serbest,18.11.2011,31,70,True
5,Ali,Diğer,11.11.2010,27,79,True
6,Veli,Diğer,11.11.2010,35,73,True
7,Ayşe,sigortacı,11.11.2010,33,79,True
8,Hüseyin,işsiz,11.11.2010,42,54,False
9,Necmi,Diğer,11.11.2010,29,92,True


Tarih formatındaki verileri parçalayarak kullanmak sıkça ihtiyaç duyduğumuz bir yöntemdir. Veri kümemizdeki Tarih özniteliğini kullanarak yıl bilgisini almak ve yeni bir öznitelik olarak eklemek istediğimiz durumda aşağıdaki kod satırlarını çalıştırabiliriz.

In [ ]:
#Tarih özniteliğindeki yıl bilgisini kullanarak 'Yıl' isimli yeni bir öznitelik oluşturuyoruz
tarih = pd.to_datetime(df['Tarih'])
df['Yıl'] = tarih.dt.year

df

,İsim,Meslek,Tarih,Yaş,Puan,Geçti,Yıl
0,Ada,işçi,11.11.2010,21,89,True,2010
1,Cem,işçi,11.11.2010,24,87,True,2010
2,Sibel,memur,11.11.2010,25,77,True,2010
3,Ahmet,serbest,18.11.2011,44,55,False,2011
4,Mehmet,serbest,18.11.2011,31,70,True,2011
5,Ali,Diğer,11.11.2010,27,79,True,2010
6,Veli,Diğer,11.11.2010,35,73,True,2010
7,Ayşe,sigortacı,11.11.2010,33,79,True,2010
8,Hüseyin,işsiz,11.11.2010,42,54,False,2010
9,Necmi,Diğer,11.11.2010,29,92,True,2010


**Kategorik Değerleri Dönüştürme (Label / One-hot Encoding)**<br>
Bilgisayar bilimlerinde kategorik verilerle çalışmak, hesaplama ve bilgisayarın bu değerleri anlaması açısından zorluklar içerir. Özellikle makine öğrenmesi modellerinin doğru çalışabilmesi için kategorik verileri, sayısal karşılıklarına (temsillerine) dönüştürmemiz gerekmektedir. Bunu yapmanın en yaygın  yolu Sklearn kütüphanesi altında yer alan LabelEncoder  kullanmaktır. Aslında bunları farklı iki yaklaşım olarak görmek mümkündür.<br>
**Label Encoder**<br>
Elimizdeki verileri direk sayısal temsillerine dönüştürmeye yarar ve kategorik her veriye sayısal bir değer atar. Genelde sadece iki değere sahip özniteliklerde kullanılır. 

In [ ]:

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder() 
df['Geçti']= label_encoder.fit_transform(df['Geçti'])
df['Meslek']=label_encoder.fit_transform(df['Meslek'])
df

,İsim,Meslek,Tarih,Yaş,Puan,Geçti,Yıl
0,Ada,2,11.11.2010,21,89,1,2010
1,Cem,2,11.11.2010,24,87,1,2010
2,Sibel,3,11.11.2010,25,77,1,2010
3,Ahmet,4,18.11.2011,44,55,0,2011
4,Mehmet,4,18.11.2011,31,70,1,2011
5,Ali,0,11.11.2010,27,79,1,2010
6,Veli,0,11.11.2010,35,73,1,2010
7,Ayşe,5,11.11.2010,33,79,1,2010
8,Hüseyin,1,11.11.2010,42,54,0,2010
9,Necmi,0,11.11.2010,29,92,1,2010


**Veri Ölçekleme<br>**
Veri ölçeklendirme ve normalize etme adımları birbirlerine benzer işler gibi görünseler de (hatta birbirleri yerine kullanılsalar da) uygulanma şekilleri farklıdır. Ölçeklendirme işleminde elimizdeki verinin sadece aralığını (range) değiştirirken (örneğin 0–1 arası ya da 1–100 arası gibi), veriyi normalize etme sürecinde verinin dağılımını normal bir dağılım olarak değiştiriyoruz.


In [ ]:
x = df[['Puan']].values.astype(float)

#Ölçeklendirme için MinMaxScaler fonksiyonunu kullanıyoruz.
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df['Puan2'] = pd.DataFrame(x_scaled)

df

,İsim,Meslek,Tarih,Yaş,Puan,Geçti,Yıl,Puan2
0,Ada,2,11.11.2010,21,89,1,2010,0.921053
1,Cem,2,11.11.2010,24,87,1,2010,0.868421
2,Sibel,3,11.11.2010,25,77,1,2010,0.605263
3,Ahmet,4,18.11.2011,44,55,0,2011,0.026316
4,Mehmet,4,18.11.2011,31,70,1,2011,0.421053
5,Ali,0,11.11.2010,27,79,1,2010,0.657895
6,Veli,0,11.11.2010,35,73,1,2010,0.500000
7,Ayşe,5,11.11.2010,33,79,1,2010,0.657895
8,Hüseyin,1,11.11.2010,42,54,0,2010,0.000000
9,Necmi,0,11.11.2010,29,92,1,2010,1.000000
